### Balanceamento

Pacotes

In [28]:
import pandas as pd
import numpy as np
from itertools import product

In [29]:
panel_data = pd.read_parquet('../Dados/pnad_covid/pnad_covid_tratada.parquet')

variáveis de identificação:
- semana_dt = semana de referencia
- domicilio_id = identificação do domicilio

variáveis exógenas:
- A003, sexo
- A004, cor
- A005, escolaridade
- grupo_etario
- formalidade
- setor
- Ocupação
- 12, 15, 17, 19, 20, 21, 22, 23, 24, 25, 27, 28, datas (semana de flexibilização em formato dummy)

variável endógena:
C013, trabalho remoto

In [30]:
panel_data = panel_data[['semana_dt', 'domicilio_id', 'sigla_uf',#indetificadores
                         'C013']] #endógena
                         #'A003', 'Raça', 'Escolaridade']] #fixas
                         #'formalidade', 'setor', 'Ocupação']] #aleatórias

In [25]:
panel_data.shape

(2508686, 4)

In [26]:
#panel_data.dropna(subset='C013', inplace=True)

In [32]:
panel_data.shape

(2508686, 4)

### Balanceamento do modelo

In [33]:
# correcting unbalanced data (like ids missing dates along the dataset)
 
ids = list(panel_data['domicilio_id'].unique())
dates = list(panel_data['semana_dt'].unique())
dates.sort()

indexes = pd.MultiIndex.from_product(iterables = [ids, dates],names = ['id', 'time'])

In [34]:
balanced_panel = product(ids, dates)

In [35]:
df_aux = pd.DataFrame(data = balanced_panel, columns=['portfolio_id', 'date'], index=indexes)

In [36]:
#merge the balanced panel with the original dataset

panel_data = pd.merge(df_aux, panel_data, how = 'left', left_on = ['portfolio_id', 'date'], right_on = ['domicilio_id', 'semana_dt'])

In [18]:
#set multiindex to the dataset with portfolio_id and date

panel_data.set_index(['portfolio_id', 'date'], inplace = True)

In [ ]:
panel_data.drop(['domicilio_id', 'semana_dt'], axis=1, inplace=True)

In [21]:
#Exporting the balanced panel

panel_data.to_parquet('../Dados/pnad_covid/pnad_covid_balanced_panel.parquet')